# Project Proposal — Heart Disease Prediction

## Introduction

Cardiovascular diseases, particularly heart disease, pose a significant global health challenge. Accurate diagnosis is crucial for effective patient care. In this proposal, we aim to address the question of diagnosing heart disease using machine learning techniques. Specifically, we will attempt to detect the presence of heart disease in patients by leveraging a heart disease dataset obtained from Kaggle. The diagnosis (num) ranges from 0 (no presence) to 4. However, our focus will be on distinguishing between the presence (values 1, 2, 3, 4) and absence (value 0) of heart disease.

The dataset consists of 16 variables, but for the purpose of this project, we will focus on a subset of 8-10 variables. Below is the description of the selected variables:

1. **Age (age) **: Age of the patient.
2. **Sex (age) **: Gender of the patient (1 = male, 0 = female).
3. **Chest Pain Type (cp)**:
   - Value 1: Typical angina
   - Value 2: Atypical angina
   - Value 3: Non-anginal pain
   - Value 4: Asymptomatic
4. **Resting Blood Pressure (trestbps)**: Measured in mm Hg upon hospital admission.
5. **Serum Cholesterol (chol)**: Serum cholesterol level in mg/dL.
6. **Fasting Blood Sugar (fbs)**:
   - 1 = Fasting blood sugar > 120 mg/dL
   - 0 = Fasting blood sugar <= 120 mg/dL
7. **Resting Electrocardiographic Results (restecg)**:
   - Value 0: Normal
   - Value 1: ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
   - Value 2: Probable or definite left ventricular hypertrophy by Estes' criteria
8. **Diagnosis of Heart Disease (num)**: Angiographic disease status, representing the degree of blockages or narrowing in major vessels.
   - Value 0: < 50% diameter narrowing
   - Value 1: > 50% diameter narrowing (in any major vessel; attributes 59 through 68 represent vessels)
   
9. **Identity (id) **: ID number of patient.

We aim to answer the following question:

#### Can K-nearest neighbors (KNN) classification accurately diagnose heart disease using patient demographics and medical histories?

### Preliminary exploratory data analysis

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')
# install.packages("cowplot")
# library(cowplot)
# library(scales)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
# install.packages("readr")
# library(readr)

In [2]:
## We are using the heart disease data. As suggested by the professor we downloaded the csv file from kaggle and uploaded it into our notebook

file_path <- "data/heart_disease_uci.csv"

heart_disease_uci <- read_csv(file_path)

# selecting only the rows we need and Change 'num' to a factor
heart_disease_data <- heart_disease_uci |> 
         select(id, age, sex, cp, trestbps, chol, fbs, restecg, num) |>
         mutate(num = as.factor(num))

# Change 'num' to a factor
# heart_disease_data$num <- factor(heart_disease_data$num, levels = c(0, 1), labels = c("No Heart Disease", "Heart Disease"))


# heart_disease_data <- heart_disease_data |>
#   mutate(
#     sex = factor(sex, levels = c(0, 1), labels = c("Female", "Male")),  # Recode 'sex'
#     cp = factor(cp, levels = c(1, 2, 3, 4), labels = c("Typical Angina", "Atypical Angina", "Non-Anginal Pain", "Asymptomatic")),  # Recode 'cp'
#     fbs = factor(fbs, levels = c(0, 1), labels = c("<= 120 mg/dL", "> 120 mg/dL")),  # Recode 'fbs'
#     restecg = factor(restecg, levels = c(0, 1, 2), labels = c("Normal", "ST-T Wave Abnormality", "Left Ventricular Hypertrophy"))  # Recode 'restecg'
#   )

# Print the first few rows of the modified dataset
head(heart_disease_data)

Rows: 920 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): sex, dataset, cp, restecg, slope, thal
dbl (8): id, age, trestbps, chol, thalch, oldpeak, ca, num
lgl (2): fbs, exang

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,age,sex,cp,trestbps,chol,fbs,restecg,num
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<fct>
1,63,Male,typical angina,145,233,TRUE,lv hypertrophy,0
2,67,Male,asymptomatic,160,286,FALSE,lv hypertrophy,2
3,67,Male,asymptomatic,120,229,FALSE,lv hypertrophy,1
4,37,Male,non-anginal,130,250,FALSE,normal,0
5,41,Female,atypical angina,130,204,FALSE,lv hypertrophy,0
6,56,Male,atypical angina,120,236,FALSE,normal,0


# Methods

We will be conducting our data analysis by examining four numerical variables, maximum heart rate achieved (thalach), resting blood pressures (trestbps), serum cholesterol (chol), and the number of major vessels coloured by fluoroscopy (ca), in order to classify and predict the diagnosis of heart disease in a patient. 

One way that we will visualize the results is by creating multiple scatterplots wherein we will first standardize each variable then distinguish them by colour/shape in order to plot data points then examine the relationships between variables. Then with these graphs, we will then be able to use K-nearest neighbors to predict new observations and further classify the presence of heart disease in a patient. 
